In [1]:
import imp, os, json, re
import service
import package.logger as logger
import package.utils as utils
import package.visionapi as visionapi
import package.reformatter as formatter
import package.evaluator as evaluator
import package.augmentation as augm

In [2]:
text_1 = """1 BENEFICIARYS DRAFT IN DUPLICATE PAYABLE AT 90 DAYS FROM BILL OF LADING DATE FOR INVOICE VALUE OF GOODS SHIPPED QUOTING LC NO. 2534IMPO00988018 AND DATE 19/06/2018.
3. FULL SET, SIGNED, CLEAN ON BOARD OCEAN BILLS OF LADING MADE OUT TO ORDER OF BANK OF BARODA, CFS PUNE, 39, MANTRI COURT, B. R. AMBEDKAR ROAD, NEAR OLD RTO OFFICE, PUNE-411001, MAHARASHTRA, INDIA MARKED FREIGHT PREPAID AND NOTIFY APPLICANT SUSHILA PARMAR INTERNATIONAL PVT LTD 31, SHREE ADINATH SHOPPING CENTRE PUNE SATARA ROAD, PUNE-411037."
"""
text_2 = """5. FULL SURVEY REPORT ISSUED BY INDEPENDENT SURVEYOR IN 1\nORIGINAL AND 1 COPY, INCLUDING:\nCERTIFICATE OF QUANTITY/WEIGHT\nCERTIFICATE OF QUALITY\nCERTIFICATE OF VESSEL TANK'S CLEANLINESS\nULLAGE REPORT\nTIME LOG\n"""
print(text_2)

5. FULL SURVEY REPORT ISSUED BY INDEPENDENT SURVEYOR IN 1
ORIGINAL AND 1 COPY, INCLUDING:
CERTIFICATE OF QUANTITY/WEIGHT
CERTIFICATE OF QUALITY
CERTIFICATE OF VESSEL TANK'S CLEANLINESS
ULLAGE REPORT
TIME LOG



In [3]:
config = utils.loadFileIfExisted('./configs/general.yaml')
req_docs = config['req_docs']
req_items = req_docs['items']
paragraph_pat = req_docs['paragraph_patterns']
quantity_pat = req_docs['quantity_patterns']

In [14]:
def testing_extraction(splitted, req_items):
    res_req_docs = {}
    for idx, line in enumerate(splitted):
        catched_list = []
        remained = []
        default_org = 0
        default_cop = 0
        for item in req_items:
            cur_key_name = item['name']
            cur_key_list = item['keys']
            for k in cur_key_list:
                contained = k.upper() in line.upper()
                if contained:
                    catched_list.append(cur_key_name)
                    break
                    
        if len(catched_list) == 0:
            remained.append(line)
        else:
            quantity_list = []
            for cur_key_name in catched_list:
                target_line = line
                target_line = utils.text2number(target_line)
                target_line = evaluator.replaceDuplicates(target_line)
                target_line = evaluator.replaceFullset(target_line, cur_key_name)
                target_line = evaluator.replaceSpecialCase(target_line)
                org_res, cop_res = evaluator.detect_quantity_with_patterns(target_line, quantity_pat['original'], quantity_pat['copy'], quantity_pat['fold'])
                quantity_list.append([org_res, cop_res])
    
            print(catched_list, quantity_list)
    return res_req_docs
        
splitted = re.split(r'\n', text_2)
res = testing_extraction(splitted, req_items)
print(json.dumps(res, indent=2))

NameError: name 'text_2' is not defined

In [15]:
import re
content = 'hello (IN ORIGINAL PLUS 1 COPY) world'
specials = ['(ORIGINAL PLUS COPY)', '(IN ORIGINAL PLUS 1 COPY)']
value = content
for case in specials:
    value = re.sub(case, '1 ORIGINAL PLUS 1 COPY', value, re.IGNORECASE)
#     value = value.replace(case, '1 ORIGINAL PLUS 1 COPY')
print(value)

hello (1 ORIGINAL PLUS 1 COPY) world
